In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='http://pokemondb.net/pokedex/all'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [3]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"#"
2:"Name"
3:"Type"
4:"Total"
5:"HP"
6:"Attack"
7:"Defense"
8:"Sp. Atk"
9:"Sp. Def"
10:"Speed"


In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


In [7]:
# Replace Headers
df = df.rename(columns={'#': 'pokedex_number', 'Name':'name', 'Type': 'type1', 'HP':'hp', 'Attack': 'attack', 'Defense':'defense', 'Sp. Atk':'sp_attack', 'Sp. Def':'sp_defense', 'Speed':'speed','Total':'total_strength_pts'})

In [8]:
df.head()

,pokedex_number,name,type1,total_strength_pts,hp,attack,defense,sp_attack,sp_defense,speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


In [9]:
# Reducing the number of pokemon to match the CVS Source's number
print(df[0:932])
df = df[0:932]
print(df.tail())

    pokedex_number                          name           type1  \
0              001                     Bulbasaur    Grass Poison   
1              002                       Ivysaur    Grass Poison   
2              003                      Venusaur    Grass Poison   
3              003        Venusaur Mega Venusaur    Grass Poison   
4              004                    Charmander           Fire    
..             ...                           ...             ...   
927            799                      Guzzlord     Dark Dragon   
928            800                      Necrozma        Psychic    
929            800   Necrozma Dusk Mane Necrozma   Psychic Steel   
930            800  Necrozma Dawn Wings Necrozma   Psychic Ghost   
931            800       Necrozma Ultra Necrozma  Psychic Dragon   

     total_strength_pts   hp  attack  defense  sp_attack  sp_defense  speed  
0                   318   45      49       49         65          65     45  
1                   405   6

In [10]:
# Seperate the types
df[['Type1', 'Type2']] = df.type1.str.split(expand=True)
print(df)

    pokedex_number                          name           type1  \
0              001                     Bulbasaur    Grass Poison   
1              002                       Ivysaur    Grass Poison   
2              003                      Venusaur    Grass Poison   
3              003        Venusaur Mega Venusaur    Grass Poison   
4              004                    Charmander           Fire    
..             ...                           ...             ...   
927            799                      Guzzlord     Dark Dragon   
928            800                      Necrozma        Psychic    
929            800   Necrozma Dusk Mane Necrozma   Psychic Steel   
930            800  Necrozma Dawn Wings Necrozma   Psychic Ghost   
931            800       Necrozma Ultra Necrozma  Psychic Dragon   

     total_strength_pts   hp  attack  defense  sp_attack  sp_defense  speed  \
0                   318   45      49       49         65          65     45   
1                   405  

In [11]:
# Drop the original combined types and re-format the new columns
df.drop(['type1'], axis=1, inplace=True)
df.drop(['total_strength_pts'], axis=1, inplace=True)

In [12]:
df = df.rename(columns={ 'Type1': 'type1', 'Type2':'type2'})

In [13]:
print(df)

    pokedex_number                          name   hp  attack  defense  \
0              001                     Bulbasaur   45      49       49   
1              002                       Ivysaur   60      62       63   
2              003                      Venusaur   80      82       83   
3              003        Venusaur Mega Venusaur   80     100      123   
4              004                    Charmander   39      52       43   
..             ...                           ...  ...     ...      ...   
927            799                      Guzzlord  223     101       53   
928            800                      Necrozma   97     107      101   
929            800   Necrozma Dusk Mane Necrozma   97     157      127   
930            800  Necrozma Dawn Wings Necrozma   97     113      109   
931            800       Necrozma Ultra Necrozma   97     167       97   

     sp_attack  sp_defense  speed    type1   type2  
0           65          65     45    Grass  Poison  
1    

In [14]:
# Replacing the 'None' values under type2 with 'No Secondary Type'
df['type2'].fillna("No Secondary Type", inplace=True)
print(df)

    pokedex_number                          name   hp  attack  defense  \
0              001                     Bulbasaur   45      49       49   
1              002                       Ivysaur   60      62       63   
2              003                      Venusaur   80      82       83   
3              003        Venusaur Mega Venusaur   80     100      123   
4              004                    Charmander   39      52       43   
..             ...                           ...  ...     ...      ...   
927            799                      Guzzlord  223     101       53   
928            800                      Necrozma   97     107      101   
929            800   Necrozma Dusk Mane Necrozma   97     157      127   
930            800  Necrozma Dawn Wings Necrozma   97     113      109   
931            800       Necrozma Ultra Necrozma   97     167       97   

     sp_attack  sp_defense  speed    type1              type2  
0           65          65     45    Grass     

In [15]:
# Remove duplicates of Pokemon having the same number ID
df = df[~df.name.str.contains("Mega")]

print(df)

    pokedex_number                          name   hp  attack  defense  \
0              001                     Bulbasaur   45      49       49   
1              002                       Ivysaur   60      62       63   
2              003                      Venusaur   80      82       83   
4              004                    Charmander   39      52       43   
5              005                    Charmeleon   58      64       58   
..             ...                           ...  ...     ...      ...   
927            799                      Guzzlord  223     101       53   
928            800                      Necrozma   97     107      101   
929            800   Necrozma Dusk Mane Necrozma   97     157      127   
930            800  Necrozma Dawn Wings Necrozma   97     113      109   
931            800       Necrozma Ultra Necrozma   97     167       97   

     sp_attack  sp_defense  speed    type1              type2  
0           65          65     45    Grass     

In [16]:
df = df[~df.name.str.contains("Dusk Mane")]
df = df[~df.name.str.contains("Dawn Wings")]
df = df[~df.name.str.contains("Ultra")]
df = df[~df.name.str.contains("Core Form")]
df = df[~df.name.str.contains("Own Tempo")]
df = df[~df.name.str.contains("Midnight")]
df = df[~df.name.str.contains("Dusk Form")]
df = df[~df.name.str.contains("School Form")]
df = df[~df.name.str.contains("Sensu Style")]
df = df[~df.name.str.contains("Pa'u Style")]
df = df[~df.name.str.contains("Pom-Pom Style")]
df = df[~df.name.str.contains("Unbound")]
df = df[~df.name.str.contains("Complete Forme")]
df = df[~df.name.str.contains("50% Form")]
df = df[~df.name.str.contains("Average Size")]
df = df[~df.name.str.contains("Large Size")]
df = df[~df.name.str.contains("Super Size")]
df = df[~df.name.str.contains("Shield Form")]
df = df[~df.name.str.contains("Female")]
df = df[~df.name.str.contains("Ash-Greninja")]
df = df[~df.name.str.contains("Aria Forme")]
df = df[~df.name.str.contains("Resolute Forme")]
df = df[~df.name.str.contains("White Kyurem")]
df = df[~df.name.str.contains("Therian Forme")]
df = df[~df.name.str.contains("Galarian")]
df = df[~df.name.str.contains("Zen Mode")]
df = df[~df.name.str.contains("Red-Striped Form")]
df = df[~df.name.str.contains("Sky Forme")]
df = df[~df.name.str.contains("Altered Forme")]
df = df[~df.name.str.contains("Heat Rotom")]
df = df[~df.name.str.contains("Wash Rotom")]
df = df[~df.name.str.contains("Frost Rotom")]
df = df[~df.name.str.contains("Fan Mane")]
df = df[~df.name.str.contains("Mow Rotom")]
df = df[~df.name.str.contains("Sandy Cloak")]
df = df[~df.name.str.contains("Trash Cloak")]
df = df[~df.name.str.contains("Attack Forme")]
df = df[~df.name.str.contains("Defense Forme")]
df = df[~df.name.str.contains("Speed Forme")]
df = df[~df.name.str.contains("Primal")]
df = df[~df.name.str.contains("Sunny Form")]
df = df[~df.name.str.contains("Rainy Form")]
df = df[~df.name.str.contains("Snowy Form")]
df = df[~df.name.str.contains("Partner")]
df = df[~df.name.str.contains("Alolan")]



print(df)

    pokedex_number        name   hp  attack  defense  sp_attack  sp_defense  \
0              001   Bulbasaur   45      49       49         65          65   
1              002     Ivysaur   60      62       63         80          80   
2              003    Venusaur   80      82       83        100         100   
4              004  Charmander   39      52       43         60          50   
5              005  Charmeleon   58      64       58         80          65   
..             ...         ...  ...     ...      ...        ...         ...   
924            796   Xurkitree   83      89       71        173          71   
925            797  Celesteela   97     101      103        107         101   
926            798     Kartana   59     181      131         59          31   
927            799    Guzzlord  223     101       53         97          53   
928            800    Necrozma   97     107      101        127          89   

     speed     type1              type2  
0       4

In [17]:
df.to_csv(r'C:\Users\Gabe\Documents\Bellevue University\Data Preparation\Final Project\Data for Pokemon Database\PokemonWebsite.csv', index = False)